## ETL del dataset output_steam_games

A continuacíon se realiza el proceso de extracción, transformación y carga del dataset output_steam_games.json

### Importaciones

In [13]:
import pandas as pd 
import ast
import json

%load_ext autoreload
%autoreload 2
import funciones

import warnings
warnings.filterwarnings("ignore")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Extracción, primera exploración y eliminación de duplicados

In [14]:
# Ruta al dataset output_steam_games
ruta_games = 'C:/Users/Gabriela/Documents/GitHub/PI_ML_OPS/data/output_steam_games.json'

# Se lee de cada línea del dataset
filas_games = []
with open(ruta_games) as f:
    for line in f.readlines():
        data = json.loads(line)
        filas_games.append(data)

# Se convierte en dataframe
df_games = pd.DataFrame(filas_games)
df_games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"


In [15]:
# Obtener información general del DataFrame
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     24083 non-null  object
 1   genres        28852 non-null  object
 2   app_name      32133 non-null  object
 3   title         30085 non-null  object
 4   url           32135 non-null  object
 5   release_date  30068 non-null  object
 6   tags          31972 non-null  object
 7   reviews_url   32133 non-null  object
 8   specs         31465 non-null  object
 9   price         30758 non-null  object
 10  early_access  32135 non-null  object
 11  id            32133 non-null  object
 12  developer     28836 non-null  object
dtypes: object(13)
memory usage: 11.9+ MB


Se observan filas que contienen  registros vacíos. Se eliminan dichos registros.

In [16]:
df_games = df_games.dropna(how='all').reset_index(drop=True)

In [17]:
#Se verifican los tipos de datos para cada columna
funciones.tipo_de_datos(df_games)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,publisher,"[<class 'str'>, <class 'float'>]",74.94,25.06,8052
1,genres,"[<class 'list'>, <class 'float'>]",89.78,10.22,3283
2,app_name,"[<class 'str'>, <class 'float'>]",99.99,0.01,2
3,title,"[<class 'str'>, <class 'float'>]",93.62,6.38,2050
4,url,[<class 'str'>],100.00,0.00,0
5,release_date,"[<class 'str'>, <class 'float'>]",93.57,6.43,2067
6,tags,"[<class 'list'>, <class 'float'>]",99.49,0.51,163
7,reviews_url,"[<class 'str'>, <class 'float'>]",99.99,0.01,2
8,specs,"[<class 'list'>, <class 'float'>]",97.92,2.08,670
9,price,"[<class 'float'>, <class 'str'>]",95.71,4.29,1377


In [19]:
# Se chequean datos duplicados en la columna 'id'
funciones.duplicados(df_games, 'id')

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
13894,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,"[Single-player, Steam Achievements, Full contr...",59.99,False,612880,Machine Games
14573,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/Wolfe...,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,"[Single-player, Steam Achievements, Full contr...",59.99,False,612880,Machine Games
74,NaN,NaN,NaN,NaN,http://store.steampowered.com/,NaN,NaN,NaN,NaN,19.99,False,NaN,NaN
30961,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",NaN,"[Single-player, Steam Achievements, Steam Trad...",19.99,False,NaN,"Rocksteady Studios,Feral Interactive (Mac)"


In [20]:
# Se eliminan los index deseados
index_a_eliminar = [14573, 74, 30961]
df_games = df_games.drop(index_a_eliminar)
# Se verifica si hay dupliacados
funciones.duplicados(df_games, 'id')

'No hay duplicados'

### Transformación

#### Transformación de la columna 'release_date'

In [21]:
# Se observan las cantidades de registros por cada fecha
df_games['release_date'].value_counts()

2012-10-16    100
2017-08-31     92
2017-09-26     89
2017-06-21     82
2017-07-25     78
             ... 
1988-04-16      1
2013-08-24      1
2011-05-07      1
2010-08-21      1
2018-10-01      1
Name: release_date, Length: 3582, dtype: int64

Se necesita extraer el año de lanzamiento del item, para ello se crea una nueva columna con el dato si existe o con un "Dato no disponible" si no esta la fecha. Luego se elimina la columa 'release_date'.

In [22]:
# Crea columna nueva con el año
df_games['release_anio'] = df_games['release_date'].apply(funciones.extraer_año)
# elimina la columna 'release_date'
df_games = df_games.drop('release_date', axis=1)
df_games.head()

,publisher,genres,app_name,title,url,tags,reviews_url,specs,price,early_access,id,developer,release_anio
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,2018
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL,2018
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com,2017
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域,2017
4,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,NaN,Dato no disponible


In [23]:
df_games['release_anio'].unique()

array(['2018', '2017', 'Dato no disponible', '1997', '1998', '2016',
       '2006', '2005', '2003', '2007', '2002', '2000', '1995', '1996',
       '1994', '2001', '1993', '2004', '1999', '2008', '2009', '1992',
       '1989', '2010', '2011', '2013', '2012', '2014', '1983', '1984',
       '2015', '1990', '1988', '1991', '1985', '1982', '1987', '1981',
       '1986', '2021', '2019', '1975', '1970', '1980'], dtype=object)

#### Transformación de la columna 'price'

En esta columna se observan valores de texto en casos de promociones o que el item es gratis. Por lo que se decide reemplazar esos valores por 0. 

In [24]:
df_games['price'] = df_games['price'].apply(funciones.convierte_float)
df_games['price'].dtype

dtype('float64')

#### Transformación de las columnas 'publisher', 'app_name', 'title', 'developer'

En estas columnas se observan valores nulos, por lo que se decide completarlos con valores del tipo 'Dato no disponible' para que el tipo de dato sea uno solo en la columna.

In [25]:
# Columnas a transformar
columnas_a_completar = ['publisher', 'app_name', 'title', 'developer']
# Se rellenan los nulos
df_relleno = df_games[columnas_a_completar].fillna('Sin dato disponible')
# Se borran las columnas originales y se concatenan las rellenas con todo el dataframe
df_games = pd.concat([df_games.drop(columnas_a_completar, axis=1), df_relleno], axis=1)
df_games.head()

,genres,url,tags,reviews_url,specs,price,early_access,id,release_anio,publisher,app_name,title,developer
0,"[Action, Casual, Indie, Simulation, Strategy]",http://store.steampowered.com/app/761140/Lost_...,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
1,"[Free to Play, Indie, RPG, Strategy]",http://store.steampowered.com/app/643980/Ironb...,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,False,643980,2018,"Making Fun, Inc.",Ironbound,Ironbound,Secret Level SRL
2,"[Casual, Free to Play, Indie, Simulation, Sports]",http://store.steampowered.com/app/670290/Real_...,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,False,670290,2017,Poolians.com,Real Pool 3D - Poolians,Real Pool 3D - Poolians,Poolians.com
3,"[Action, Adventure, Casual]",http://store.steampowered.com/app/767400/2222/,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,2017,彼岸领域,弹炸人2222,弹炸人2222,彼岸领域
4,NaN,http://store.steampowered.com/app/773570/Log_C...,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,Dato no disponible,Sin dato disponible,Log Challenge,Sin dato disponible,Sin dato disponible


#### Transformación columna 'genres'

Se necesita hacer una columna con cada género.

In [26]:
df_games = df_games.explode('genres')
df_games = df_games.dropna(subset=['genres'])
df_games.head()

,genres,url,tags,reviews_url,specs,price,early_access,id,release_anio,publisher,app_name,title,developer
0,Action,http://store.steampowered.com/app/761140/Lost_...,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
0,Casual,http://store.steampowered.com/app/761140/Lost_...,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
0,Indie,http://store.steampowered.com/app/761140/Lost_...,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
0,Simulation,http://store.steampowered.com/app/761140/Lost_...,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
0,Strategy,http://store.steampowered.com/app/761140/Lost_...,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro


#### Columnas 'tags', 'specs', 'url' y 'reviews_url'

Estas columnas, en esta primer instancia, no se utilizarán para las consultas de la API ni para el modelo de recomendación, por lo tanto se eliminan del dataframe.

In [27]:
df_games = df_games.drop(['tags', 'specs', 'url', 'reviews_url'], axis=1)
df_games.columns

Index(['genres', 'price', 'early_access', 'id', 'release_anio', 'publisher',
       'app_name', 'title', 'developer'],
      dtype='object')

In [28]:
# Se verifica como quedan los tipos de datos y si hay nulos
funciones.tipo_de_datos(df_games)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,genres,[<class 'str'>],100.0,0.0,0
1,price,[<class 'float'>],100.0,0.0,0
2,early_access,[<class 'bool'>],100.0,0.0,0
3,id,[<class 'str'>],100.0,0.0,0
4,release_anio,[<class 'str'>],100.0,0.0,0
5,publisher,[<class 'str'>],100.0,0.0,0
6,app_name,[<class 'str'>],100.0,0.0,0
7,title,[<class 'str'>],100.0,0.0,0
8,developer,[<class 'str'>],100.0,0.0,0


### Carga del dataset output_steam_games.json

Se guarda el archivo con el dataset limpio en formato .csv

In [29]:
archivo_limpio = 'steam_games_limpio.csv'
df_games.to_csv(archivo_limpio, index=False, encoding='utf-8')
print(f'Se guardó el archivo {archivo_limpio}')

Se guardó el archivo steam_games_limpio.csv
